In [1]:
import pandas as pd

feature_names =  ['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', 'magX', 'magY', 'magZ']

cat_dict = {'1a': 1,'1b': 2,'2a': 3, '2b': 4, '2c': 5, '2d': 6, '3a': 7, '3b': 8, 'TRANSITION': 0}

def process_file(filename):
    col_names = ['timestamp'] + feature_names + ['cat']
    df = pd.read_csv(filename, header=None, names=col_names)
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.dropna()
    df['is_transition'] = df['cat'] == "TRANSITION"
    df['cat'] = df['cat'].map(cat_dict) 
    return df

df1 = process_file('1_50Hz.csv')
df2 = process_file('2_50Hz.csv')
df3 = process_file('3_50Hz.csv')

In [2]:
df1_filtered = df1[df1.is_transition == False]
df2_filtered = df2[df2.is_transition == False]
df3_filtered = df3[df3.is_transition == False]

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0-beta1


In [4]:
def fit_model(dataset):
    model = keras.Sequential([
        keras.layers.Dense(128, activation=tf.nn.relu),
        keras.layers.Dense(64, activation=tf.nn.relu),
        keras.layers.Dense(16, activation=tf.nn.relu),
        keras.layers.Dense(9, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    X = dataset[feature_names].values
    y = dataset.cat.values

    model.fit(X, y, epochs=8)
    return model

In [5]:
model1 = fit_model(df1)
model2 = fit_model(df2)
model3 = fit_model(pd.concat([df1, df2]))

W0807 21:26:50.635231  2488 deprecation.py:323] From C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 803002 samples
Epoch 1/8
803002/803002 [==============================] - 47s 58us/sample - loss: 0.5207 - accuracy: 0.8156
Epoch 2/8
803002/803002 [==============================] - 36s 45us/sample - loss: 0.3153 - accuracy: 0.8961
Epoch 3/8
803002/803002 [==============================] - 34s 43us/sample - loss: 0.2696 - accuracy: 0.9105
Epoch 4/8
803002/803002 [==============================] - 34s 43us/sample - loss: 0.2489 - accuracy: 0.9165
Epoch 5/8
803002/803002 [==============================] - 37s 46us/sample - loss: 0.2350 - accuracy: 0.9207
Epoch 6/8
803002/803002 [==============================] - 35s 44us/sample - loss: 0.2250 - accuracy: 0.9236
Epoch 7/8
803002/803002 [==============================] - 35s 43us/sample - loss: 0.2168 - accuracy: 0.9262
Epoch 8/8
803002/803002 [==============================] - 35s 44us/sample - loss: 0.2107 - accuracy: 0.9280
Train on 846331 samples
Epoch 1/8
846331/846331 [==============================] - 36s 43us/sample - los

In [7]:
def evaluate(model, testset):
    test_loss, test_acc = model.evaluate(testset[feature_names].values, testset.cat)
    print('Test accuracy:', test_acc)

evaluate(model1, df3)
evaluate(model2, df3)
evaluate(model3, df3)
evaluate(model1, df3_filtered)
evaluate(model2, df3_filtered)
evaluate(model3, df3_filtered)

888477/888477 [==============================] - 13s 14us/sample - loss: 9.0070 - accuracy: 0.1643
Test accuracy: 0.16425073
888477/888477 [==============================] - 15s 16us/sample - loss: 5.7836 - accuracy: 0.2134
Test accuracy: 0.21336624
888477/888477 [==============================] - 14s 16us/sample - loss: 6.0341 - accuracy: 0.2159
Test accuracy: 0.21592906
678459/678459 [==============================] - 11s 16us/sample - loss: 10.0930 - accuracy: 0.1900
Test accuracy: 0.19000264
678459/678459 [==============================] - 11s 17us/sample - loss: 6.8769 - accuracy: 0.1299
Test accuracy: 0.12985899
678459/678459 [==============================] - 11s 17us/sample - loss: 6.6361 - accuracy: 0.1717
Test accuracy: 0.17166108
